In [1]:
#@title 🏥 B病院 日次採血患者数予測アプリ (Google Drive版)
# =================================================================
# 1. ライブラリのインストール
# =================================================================
!pip -q install ipywidgets jpholiday scikit-learn xgboost pandas

# =================================================================
# 2. ライブラリのインポート
# =================================================================
import pandas as pd
import numpy as np
import jpholiday
from datetime import date, timedelta
import joblib
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# =================================================================
# 3. Google Driveの連携
# =================================================================
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"エラー: Google Driveの連携に失敗しました。{e}")

# =================================================================
# 4. 予測用の特徴量を作成する関数
# =================================================================
def create_features_for_pred(target_date, patient_count, weather, feature_columns):
    """UIからの入力値を受け取り、学習時と同じ形式のデータフレームを返す"""
    target_dt = pd.to_datetime(target_date)

    is_holiday = (jpholiday.is_holiday(target_dt) or target_dt.weekday() >= 5 or
                  (target_dt.month == 12 and target_dt.day >= 29) or (target_dt.month == 1 and target_dt.day <= 3))
    prev_dt = target_dt - timedelta(days=1)
    is_prev_holiday = (jpholiday.is_holiday(prev_dt) or prev_dt.weekday() >= 5 or
                       (prev_dt.month == 12 and prev_dt.day >= 29) or (prev_dt.month == 1 and prev_dt.day <= 3))

    pred_df = pd.DataFrame(columns=feature_columns)
    pred_df.loc[0] = 0

    # Ensure all numeric columns from training are present and handled
    numeric_cols = ['降水量', '平均気温', '最高気温', '最低気温', '平均湿度', '平均風速']
    for col in numeric_cols:
        if col in pred_df.columns:
            # In a real UI, you might not have future weather data.
            # Here, we'll use a neutral value (0) or you could use seasonal averages.
            # For simplicity, we are not adding them as inputs in the UI.
            # The model learned their patterns, but for prediction, we create the columns.
            pred_df[col] = 0 # Assuming neutral effect if not provided

    if '月' in pred_df.columns: pred_df['月'] = target_dt.month
    if '週回数' in pred_df.columns: pred_df['週回数'] = (target_dt.day - 1) // 7 + 1
    if '前日祝日フラグ' in pred_df.columns: pred_df['前日祝日フラグ'] = int(is_prev_holiday)
    if 'total_outpatient_count' in pred_df.columns: pred_df['total_outpatient_count'] = patient_count
    if '雨フラグ' in pred_df.columns: pred_df['雨フラグ'] = 1 if weather == '雨' else 0
    if '雪フラグ' in pred_df.columns: pred_df['雪フラグ'] = 1 if weather == '雪' else 0

    weekday_map = {0: '曜日_月', 1: '曜日_火', 2: '曜日_水', 3: '曜日_木', 4: '曜日_金', 5: '曜日_土'}
    weekday_col = weekday_map.get(target_dt.weekday())
    if weekday_col in pred_df.columns:
        pred_df[weekday_col] = 1

    # Fill any remaining NaN that might result from column creation
    pred_df.fillna(0, inplace=True)

    return pred_df[feature_columns]

# =================================================================
# 5. アプリケーション本体
# =================================================================
try:
    print("\n--- 🏥 B病院 日次採血患者数予測アプリ ---")

    # --- ファイルパスをGoogle Driveに指定 ---
    DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Colab_Data/Hospital_B/' # B病院用のパスに変更
    MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_daily_finetuned.json'      # B病院用のモデル名に変更
    SCALER_PATH = DRIVE_FOLDER_PATH + 'model_B_daily_scaler.joblib'    # B病院用のスケーラー名に変更

    # 1. モデルとスケーラーの読み込み
    model = xgb.XGBRegressor()
    model.load_model(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)

    print("\n✅ モデル読込完了。予測を開始できます。")

    # 2. UIウィジェットの作成
    date_picker = widgets.DatePicker(description='予測日', value=date.today() + timedelta(days=1))
    patient_input = widgets.IntText(value=1000, description='延べ外来患者数:') # B病院の平均値に合わせるとより良い
    weather_input = widgets.RadioButtons(options=['晴', '曇', '雨', '雪'], value='晴', description='天気:')
    predict_button = widgets.Button(description='予測実行', button_style='success', icon='calculator')
    output_area = widgets.Output()

    # 3. ボタンクリック時の処理
    def on_predict_clicked(b):
        with output_area:
            clear_output()
            # Create the feature vector using the function
            input_df = create_features_for_pred(date_picker.value, patient_input.value, weather_input.value, scaler.feature_names_in_)

            # Use the model to predict
            prediction = model.predict(input_df)

            # Display the result
            print(f"--- 予測結果 ({date_picker.value}) ---")
            print(f"📅 条件: 外来患者 {patient_input.value}人, 天気 {weather_input.value}")
            print("---------------------------------")
            print(f"🚀 予測採血患者数: 約 {prediction[0]:.0f}人")

    predict_button.on_click(on_predict_clicked)

    # 4. UIの表示
    display(date_picker, patient_input, weather_input, predict_button, output_area)

except FileNotFoundError:
    print(f"エラー: 指定されたパスにファイルが見つかりません。")
    print(f"  - 確認したモデルパス: {MODEL_PATH}")
    print(f"  - 確認したスケーラーパス: {SCALER_PATH}")
    print("\nGoogle Driveのフォルダ構成と、コード内の'DRIVE_FOLDER_PATH'が一致しているか確認してください。")
except Exception as e:
    print(f"\n予期せぬエラーが発生しました: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.8 MB/s eta 0:00:00
Mounted at /content/drive

--- 🏥 B病院 日次採血患者数予測アプリ ---

✅ モデル読込完了。予測を開始できます。


DatePicker(value=datetime.date(2025, 12, 5), description='予測日')

IntText(value=1000, description='延べ外来患者数:')

RadioButtons(description='天気:', options=('晴', '曇', '雨', '雪'), value='晴')

Button(button_style='success', description='予測実行', icon='calculator', style=ButtonStyle())

Output()

In [ ]:
#@title 🏥 B病院 採血患者数予測アプリ (5営業日・空白スキップ対応版)
# =================================================================
# 1. ライブラリのインストール
# =================================================================
!pip -q install ipywidgets jpholiday scikit-learn xgboost pandas

# =================================================================
# 2. ライブラリのインポート
# =================================================================
import pandas as pd
import numpy as np
import jpholiday
from datetime import date, timedelta
import joblib
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive

# =================================================================
# 3. Google Driveの連携
# =================================================================
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"エラー: Google Driveの連携に失敗しました。{e}")

# =================================================================
# 4. ヘルパー関数
# =================================================================
def create_features_for_pred(target_date, patient_count, weather, feature_columns):
    """UIからの入力値を受け取り、学習時と同じ形式のデータフレームを返す"""
    target_dt = pd.to_datetime(target_date)
    is_holiday = (jpholiday.is_holiday(target_dt) or target_dt.weekday() >= 5 or
                  (target_dt.month == 12 and target_dt.day >= 29) or (target_dt.month == 1 and target_dt.day <= 3))
    prev_dt = target_dt - timedelta(days=1)
    is_prev_holiday = (jpholiday.is_holiday(prev_dt) or prev_dt.weekday() >= 5 or
                       (prev_dt.month == 12 and prev_dt.day >= 29) or (prev_dt.month == 1 and prev_dt.day <= 3))

    pred_df = pd.DataFrame(columns=feature_columns)
    pred_df.loc[0] = 0

    # 学習時に使われた全ての気象関連カラムを生成
    numeric_cols = ['降水量', '平均気温', '最高気温', '最低気温', '平均湿度', '平均風速']
    for col in numeric_cols:
        if col in pred_df.columns:
            pred_df[col] = 0 # UIからは入力しないため、中立的な値(0)を仮入力

    if '月' in pred_df.columns: pred_df['月'] = target_dt.month
    if '週回数' in pred_df.columns: pred_df['週回数'] = (target_dt.day - 1) // 7 + 1
    if '前日祝日フラグ' in pred_df.columns: pred_df['前日祝日フラグ'] = int(is_prev_holiday)
    if 'total_outpatient_count' in pred_df.columns: pred_df['total_outpatient_count'] = patient_count
    if '雨フラグ' in pred_df.columns: pred_df['雨フラグ'] = 1 if weather == '雨' else 0
    if '雪フラグ' in pred_df.columns: pred_df['雪フラグ'] = 1 if weather == '雪' else 0

    weekday_map = {0: '曜日_月', 1: '曜日_火', 2: '曜日_水', 3: '曜日_木', 4: '曜日_金', 5: '曜日_土'}
    weekday_col = weekday_map.get(target_dt.weekday())
    if weekday_col in pred_df.columns:
        pred_df[weekday_col] = 1

    pred_df.fillna(0, inplace=True)
    return pred_df[feature_columns]

def get_next_n_business_days(start_date, n):
    """開始日から次のn営業日を計算してリストで返す"""
    business_days = []
    current_date = pd.to_datetime(start_date)
    while len(business_days) < n:
        is_business_day = not (jpholiday.is_holiday(current_date) or current_date.weekday() >= 5 or \
                               (current_date.month == 12 and current_date.day >= 29) or \
                               (current_date.month == 1 and current_date.day <= 3))
        if is_business_day:
            business_days.append(current_date)
        current_date += timedelta(days=1)
    return business_days

# =================================================================
# 5. アプリケーション本体
# =================================================================
try:
    print("\n--- 🏥 B病院 採血患者数予測アプリ (5営業日予測版) ---")

    # --- ファイルパスをGoogle Driveに指定 ---
    DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Colab_Data/Hospital_B/'
    MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_daily_finetuned.json'
    SCALER_PATH = DRIVE_FOLDER_PATH + 'model_B_daily_scaler.joblib'

    # 1. モデルとスケーラーの読み込み
    model = xgb.XGBRegressor(); model.load_model(MODEL_PATH)
    scaler = joblib.load(SCALER_PATH)

    print("\n✅ モデル読込完了。予測を開始できます。")

    # --- 2. UIウィジェットの作成 ---
    start_date_picker = widgets.DatePicker(description='予測開始日', value=date.today() + timedelta(days=1))
    patient_inputs = [widgets.Text(value='1000', description='') for _ in range(5)] # B病院用のデフォルト値
    predict_button = widgets.Button(description='予測実行', button_style='success', icon='calendar')
    output_area = widgets.Output()

    # --- 3. 動的なUI更新処理 ---
    def update_input_labels(change):
        start_date = pd.to_datetime(change['new'])
        business_days = get_next_n_business_days(start_date, 5)
        weekday_jp = ['月', '火', '水', '木', '金', '土', '日']
        for i, day in enumerate(business_days):
            patient_inputs[i].description = f"{day.strftime('%m/%d')} ({weekday_jp[day.weekday()]}):"

    # --- 4. ボタンクリック時の処理 ---
    def on_predict_clicked(b):
        with output_area:
            clear_output(wait=True)
            start_date = pd.to_datetime(start_date_picker.value)
            business_days = get_next_n_business_days(start_date, 5)
            results = []

            for i in range(5):
                if patient_inputs[i].value.strip():
                    try:
                        patient_count = int(patient_inputs[i].value)
                        current_date = business_days[i]
                        # 常に「晴れ」の条件で予測
                        input_df = create_features_for_pred(current_date, patient_count, '晴', scaler.feature_names_in_)
                        prediction = model.predict(input_df)

                        results.append({
                            '日付': current_date.strftime('%Y-%m-%d'),
                            '曜日': ['月', '火', '水', '木', '金', '土', '日'][current_date.weekday()],
                            '延べ外来患者数': patient_count,
                            '予測採血患者数': int(round(prediction[0]))
                        })
                    except ValueError:
                        pass # 数字以外の入力はスキップ

            if results:
                result_df = pd.DataFrame(results)
                print("--- 予測結果 ---")
                display(result_df)
            else:
                print("延べ外来患者数が入力されていないため、予測は実行されませんでした。")

    # --- 5. UIの初期化と表示 ---
    predict_button.on_click(on_predict_clicked)
    start_date_picker.observe(update_input_labels, names='value')
    update_input_labels({'new': start_date_picker.value}) # 初期ラベルを設定

    display(start_date_picker)
    print("--- 各営業日の延べ外来患者数を入力してください (空白の場合は予測しません) ---")
    for widget in patient_inputs:
        display(widget)
    display(predict_button, output_area)

except FileNotFoundError:
    print(f"エラー: 指定されたパスにファイルが見つかりません。")
    print(f"  - 確認したモデルパス: {MODEL_PATH}")
    print(f"  - 確認したスケーラーパス: {SCALER_PATH}")
    print("\nGoogle Driveのフォルダ構成と、コード内の'DRIVE_FOLDER_PATH'が一致しているか確認してください。")
except Exception as e:
    print(f"\n予期せぬエラーが発生しました: {e}")

Mounted at /content/drive

--- 🏥 B病院 採血患者数予測アプリ (5営業日予測版) ---

✅ モデル読込完了。予測を開始できます。


DatePicker(value=datetime.date(2025, 9, 25), description='予測開始日')

--- 各営業日の延べ外来患者数を入力してください (空白の場合は予測しません) ---


Text(value='1000', description='09/25 (木):')

Text(value='1000', description='09/26 (金):')

Text(value='1000', description='09/29 (月):')

Text(value='1000', description='09/30 (火):')

Text(value='1000', description='10/01 (水):')

Button(button_style='success', description='予測実行', icon='calendar', style=ButtonStyle())

Output()

In [ ]:
#@title 🏥 B病院 時間帯別 採血患者数予測アプリ
# =================================================================
# 1. ライブラリのインストール
# =================================================================
!pip -q install ipywidgets jpholiday scikit-learn xgboost pandas japanize-matplotlib

# =================================================================
# 2. ライブラリのインポート
# =================================================================
import pandas as pd
import numpy as np
import jpholiday
from datetime import date, timedelta, time
import json
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive
import matplotlib.pyplot as plt
import japanize_matplotlib

# =================================================================
# 3. Google Driveの連携
# =================================================================
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"エラー: Google Driveの連携に失敗しました。{e}")

# =================================================================
# 4. アプリケーション本体
# =================================================================
try:
    print("\n--- 🏥 B病院 時間帯別 採血患者数予測アプリ ---")

    # --- 4a. ファイルパスの定義 ---
    DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Colab_Data/Hospital_B/' # B病院用のパスに変更
    MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_timeseries_finetuned.json'      # B病院用のモデル名に変更
    COLUMNS_PATH = DRIVE_FOLDER_PATH + 'columns_B_timeseries.json'    # B病院用のカラムリスト名に変更

    # --- 4b. モデルとカラムリストの読み込み ---
    model = xgb.XGBRegressor(); model.load_model(MODEL_PATH)
    with open(COLUMNS_PATH, 'r') as f:
        feature_columns = json.load(f)

    print("\n✅ モデル読込完了。")

    # --- 4c. UIウィジェットの作成 ---
    date_picker = widgets.DatePicker(description='予測対象日', value=date.today() + timedelta(days=1))
    patient_input = widgets.IntText(value=1000, description='延べ外来患者数:') # B病院用のデフォルト値
    weather_input = widgets.Dropdown(options=['晴', '曇', '雨', '雪', '快晴', '薄曇'], value='晴', description='天気予報:')
    predict_button = widgets.Button(description='明日のスケジュールを予測', button_style='success', icon='chart-line')
    output_area = widgets.Output()

    # --- 4d. 予測実行ロジック ---
    def on_predict_clicked(b):
        with output_area:
            clear_output(wait=True)
            print("予測計算中...")

            target_date = pd.to_datetime(date_picker.value)
            total_patients = patient_input.value
            weather = weather_input.value

            time_slots = pd.date_range(start=target_date.replace(hour=8), end=target_date.replace(hour=18), freq='30T')
            predictions = []

            lags = {'lag_30min': 0.0, 'lag_60min': 0.0, 'lag_90min': 0.0}
            rolling_mean = 0.0

            is_holiday_daily = jpholiday.is_holiday(target_date) or target_date.weekday() >= 5 or \
                               (target_date.month == 12 and target_date.day >= 29) or (target_date.month == 1 and target_date.day <= 3)
            prev_date = target_date - timedelta(days=1)
            is_prev_holiday = jpholiday.is_holiday(prev_date) or prev_date.weekday() >= 5 or \
                              (prev_date.month == 12 and prev_date.day >= 29) or (prev_date.month == 1 and prev_date.day <= 3)

            for ts in time_slots:
                features = pd.DataFrame(columns=feature_columns); features.loc[0] = 0

                features['hour'] = ts.hour
                features['minute'] = ts.minute
                features['is_first_slot'] = 1 if (ts.hour == 8 and ts.minute == 0) else 0
                features['is_second_slot'] = 1 if (ts.hour == 8 and ts.minute == 30) else 0
                if '月' in features.columns: features['月'] = ts.month
                if '週回数' in features.columns: features['週回数'] = (ts.day - 1) // 7 + 1
                if '前日祝日フラグ' in features.columns: features['前日祝日フラグ'] = int(is_prev_holiday)
                features['total_outpatient_count'] = total_patients
                features['is_holiday'] = int(is_holiday_daily)
                features['雨フラグ'] = 1 if '雨' in weather else 0
                features['雪フラグ'] = 1 if '雪' in weather else 0
                weather_cat_col = f"天気カテゴリ_{weather[0]}"
                if weather_cat_col in features.columns:
                    features[weather_cat_col] = 1
                dayofweek_col = f"dayofweek_{ts.dayofweek}"
                if dayofweek_col in features.columns:
                    features[dayofweek_col] = 1

                rolling_mean = (lags['lag_30min'] + lags['lag_60min']) / 2.0
                for lag_col, lag_val in lags.items():
                    if lag_col in features.columns: features[lag_col] = lag_val
                if 'rolling_mean_60min' in features.columns: features['rolling_mean_60min'] = rolling_mean

                prediction = max(0, round(model.predict(features[feature_columns])[0]))
                predictions.append({'時間帯': ts.strftime('%H:%M'), '予測患者数': prediction})

                new_lags = lags.copy()
                new_lags['lag_90min'] = new_lags['lag_60min']
                new_lags['lag_60min'] = new_lags['lag_30min']
                new_lags['lag_30min'] = prediction
                lags = new_lags

            clear_output(wait=True)
            result_df = pd.DataFrame(predictions)
            print(f"--- {target_date.strftime('%Y-%m-%d')} の予測結果 (合計: {result_df['予測患者数'].sum()}人) ---")

            fig, ax = plt.subplots(figsize=(12, 5))
            result_df.plot(kind='bar', x='時間帯', y='予測患者数', ax=ax, legend=None, width=0.8)
            ax.set_title(f"{target_date.strftime('%Y-%m-%d')} の時間帯別 予測患者数", fontsize=16)
            ax.set_ylabel('予測患者数 (人)')
            ax.set_xlabel('時間帯')
            ax.tick_params(axis='x', rotation=45)
            for p in ax.patches:
                ax.annotate(str(int(p.get_height())), (p.get_x() * 1.005, p.get_height() * 1.005))

            plt.tight_layout(); plt.show()
            display(result_df)

    predict_button.on_click(on_predict_clicked)

    # --- 4e. UIの表示 ---
    display(date_picker, patient_input, weather_input, predict_button, output_area)

except FileNotFoundError:
    print(f"エラー: 指定されたパスにファイルが見つかりません。")
    print(f"  - 確認したモデルパス: {MODEL_PATH}")
    print(f"  - 確認したカラムパス: {COLUMNS_PATH}")
    print("\nGoogle Driveのフォルダ構成と、コード内の'DRIVE_FOLDER_PATH'が一致しているか確認してください。")
except Exception as e:
    print(f"\n予期せぬエラーが発生しました: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 57.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Mounted at /content/drive

--- 🏥 B病院 時間帯別 採血患者数予測アプリ ---

✅ モデル読込完了。


DatePicker(value=datetime.date(2025, 9, 25), description='予測対象日')

IntText(value=1000, description='延べ外来患者数:')

Dropdown(description='天気予報:', options=('晴', '曇', '雨', '雪', '快晴', '薄曇'), value='晴')

Button(button_style='success', description='明日のスケジュールを予測', icon='chart-line', style=ButtonStyle())

Output()

In [ ]:
#@title 🏥 B病院 待ち人数・待ち時間 統合予測アプリ
# =================================================================
# 1. ライブラリのインストール
# =================================================================
# !pip -q install ipywidgets jpholiday scikit-learn xgboost pandas japanize-matplotlib

# =================================================================
# 2. ライブラリのインポート
# =================================================================
import pandas as pd
import numpy as np
import jpholiday
from datetime import date, timedelta, time
import json
import xgboost as xgb
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import drive
import matplotlib.pyplot as plt
import japanize_matplotlib

# =================================================================
# 3. Google Driveの連携
# =================================================================
try:
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"エラー: Google Driveの連携に失敗しました。{e}")

# =================================================================
# 4. アプリケーション本体
# =================================================================
try:
    print("\n--- 🏥 B病院 待ち人数・待ち時間 統合予測アプリ ---")

    # --- 4a. ファイルパスの定義 ---
    DRIVE_FOLDER_PATH = '/content/drive/MyDrive/Colab_Data/Hospital_B/'
    # モデル1: 受付人数予測モデル (B病院ファインチューニング版)
    COUNT_MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_timeseries_finetuned.json'
    COUNT_COLUMNS_PATH = DRIVE_FOLDER_PATH + 'columns_B_timeseries.json'
    # モデル2&3: 待ち時間・待ち人数予測モデル (B病院ファインチューニング版)
    WAITTIME_MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_waittime_30min_ft.json'
    QUEUE_MODEL_PATH = DRIVE_FOLDER_PATH + 'model_B_queue_30min_ft.json'
    MULTI_COLUMNS_PATH = DRIVE_FOLDER_PATH + 'columns_B_multi_30min.json'

    # --- 4b. 全モデルとカラムリストの読み込み ---
    count_model = xgb.XGBRegressor(); count_model.load_model(COUNT_MODEL_PATH)
    with open(COUNT_COLUMNS_PATH, 'r') as f: count_feature_columns = json.load(f)
    waittime_model = xgb.XGBRegressor(); waittime_model.load_model(WAITTIME_MODEL_PATH)
    queue_model = xgb.XGBRegressor(); queue_model.load_model(QUEUE_MODEL_PATH)
    with open(MULTI_COLUMNS_PATH, 'r') as f: multi_feature_columns = json.load(f)
    print("\n✅ B病院専用の3種類の予測モデルの読込完了。")

    # --- 4c. UIウィジェットの作成 ---
    date_picker = widgets.DatePicker(description='予測対象日', value=date.today() + timedelta(days=1))
    patient_input = widgets.IntText(value=1000, description='延べ外来患者数:')
    weather_input = widgets.Dropdown(options=['晴', '曇', '雨', '雪', '快晴', '薄曇'], value='晴', description='天気予報:')
    predict_button = widgets.Button(description='明日の状況をシミュレート', button_style='success', icon='cogs')
    output_area = widgets.Output()

    # --- 4d. 予測実行ロジック ---
    def on_predict_clicked(b):
        with output_area:
            clear_output(wait=True)
            print("シミュレーション計算中...")

            target_date = pd.to_datetime(date_picker.value)
            total_patients = patient_input.value
            weather = weather_input.value

            is_holiday_daily = jpholiday.is_holiday(target_date) or target_date.weekday() >= 5
            prev_date = target_date - timedelta(days=1)
            is_prev_holiday = jpholiday.is_holiday(prev_date) or prev_date.weekday() >= 5

            time_slots = pd.date_range(start=target_date.replace(hour=8), end=target_date.replace(hour=18), freq='30T')
            results = []
            lags = {'lag_30min': 0.0, 'lag_60min': 0.0, 'lag_90min': 0.0}
            queue_at_start = 0

            for ts in time_slots:
                # --- 1. 受付人数を予測 ---
                count_features = pd.DataFrame(columns=count_feature_columns); count_features.loc[0] = 0
                # (受付人数予測用の特徴量を作成)
                count_features['hour'] = ts.hour; count_features['minute'] = ts.minute
                count_features['is_first_slot'] = 1 if (ts.hour == 8 and ts.minute == 0) else 0
                count_features['is_second_slot'] = 1 if (ts.hour == 8 and ts.minute == 30) else 0
                if '月' in count_features.columns: count_features['月'] = ts.month
                if '週回数' in count_features.columns: count_features['週回数'] = (ts.day - 1) // 7 + 1
                if '前日祝日フラグ' in count_features.columns: count_features['前日祝日フラグ'] = int(is_prev_holiday)
                count_features['total_outpatient_count'] = total_patients
                count_features['is_holiday'] = int(is_holiday_daily)
                weather_cat_col = f"天気カテゴリ_{weather[0]}"
                if weather_cat_col in count_features.columns: count_features[weather_cat_col] = 1
                dayofweek_col = f"dayofweek_{ts.dayofweek}"
                if dayofweek_col in count_features.columns: count_features[dayofweek_col] = 1
                rolling_mean = (lags['lag_30min'] + lags['lag_60min']) / 2.0
                for lag_col, lag_val in lags.items():
                    if lag_col in count_features.columns: count_features[lag_col] = lag_val
                if 'rolling_mean_60min' in count_features.columns: count_features['rolling_mean_60min'] = rolling_mean

                predicted_reception_count = max(0, int(round(count_model.predict(count_features[count_feature_columns])[0])))

                # --- 2. 待ち人数と待ち時間を予測 ---
                multi_features = pd.DataFrame(columns=multi_feature_columns); multi_features.loc[0] = 0
                # (待ち時間・待ち人数予測用の特徴量を作成)
                multi_features['hour'] = ts.hour
                multi_features['minute'] = ts.minute
                multi_features['reception_count'] = predicted_reception_count
                multi_features['queue_at_start_of_slot'] = queue_at_start
                if '月' in multi_features.columns: multi_features['月'] = ts.month
                if '週回数' in multi_features.columns: multi_features['週回数'] = (ts.day - 1) // 7 + 1
                if '前日祝日フラグ' in multi_features.columns: multi_features['前日祝日フラグ'] = int(is_prev_holiday)
                multi_features['total_outpatient_count'] = total_patients
                multi_features['is_holiday'] = int(is_holiday_daily)
                weather_cat_col_multi = f"天気カテゴリ_{weather[0]}"
                if weather_cat_col_multi in multi_features.columns: multi_features[weather_cat_col_multi] = 1
                dayofweek_col_multi = f"dayofweek_{ts.dayofweek}"
                if dayofweek_col_multi in multi_features.columns: multi_features[dayofweek_col_multi] = 1

                predicted_queue_size = max(0, int(round(queue_model.predict(multi_features[multi_feature_columns])[0])))
                predicted_avg_wait_time = max(0, int(round(waittime_model.predict(multi_features[multi_feature_columns])[0])))

                # --- 3. 結果の保存と状態の更新 ---
                results.append({
                    '時間帯': ts.strftime('%H:%M'),
                    '予測受付数': predicted_reception_count,
                    '予測待ち人数(人)': predicted_queue_size,
                    '予測平均待ち時間(分)': predicted_avg_wait_time
                })
                lags = {'lag_30min': predicted_reception_count, 'lag_60min': lags['lag_30min'], 'lag_90min': lags['lag_60min']}
                queue_at_start = predicted_queue_size

            # --- 結果の表示 ---
            clear_output(wait=True)
            result_df = pd.DataFrame(results)
            print(f"--- {target_date.strftime('%Y-%m-%d')} の予測シミュレーション結果 ---")

            fig, ax1 = plt.subplots(figsize=(15, 6))
            bars = ax1.bar(result_df['時間帯'], result_df['予測待ち人数(人)'], color='skyblue', label='予測待ち人数')
            ax1.set_xlabel('時間帯', fontsize=12)
            ax1.set_ylabel('予測待ち人数 (人)', color='blue', fontsize=12)
            ax1.tick_params(axis='y', labelcolor='blue'); ax1.tick_params(axis='x', rotation=45)
            for bar in bars:
                yval = bar.get_height()
                ax1.text(bar.get_x() + bar.get_width()/2.0, yval, int(yval), va='bottom', ha='center')

            ax2 = ax1.twinx()
            ax2.plot(result_df['時間帯'], result_df['予測平均待ち時間(分)'], color='red', marker='o', label='予測平均待ち時間')
            ax2.set_ylabel('予測平均待ち時間 (分)', color='red', fontsize=12)
            ax2.tick_params(axis='y', labelcolor='red')
            for i, txt in enumerate(result_df['予測平均待ち時間(分)']):
                ax2.annotate(txt, (result_df['時間帯'][i], result_df['予測平均待ち時間(分)'][i]), textcoords="offset points", xytext=(0,5), ha='center')

            plt.title(f"{target_date.strftime('%Y-%m-%d')} の予測待ち人数と平均待ち時間", fontsize=16)
            fig.tight_layout(); plt.show()
            display(result_df)

    predict_button.on_click(on_predict_clicked)

    # --- 4e. UIの表示 ---
    display(date_picker, patient_input, weather_input, predict_button, output_area)

except FileNotFoundError:
    print(f"エラー: 必要なファイルが見つかりません。パスを確認してください。")
except Exception as e:
    print(f"\n予期せぬエラーが発生しました: {e}")

Mounted at /content/drive

--- 🏥 B病院 待ち人数・待ち時間 統合予測アプリ ---

✅ B病院専用の3種類の予測モデルの読込完了。


DatePicker(value=datetime.date(2025, 9, 25), description='予測対象日')

IntText(value=1000, description='延べ外来患者数:')

Dropdown(description='天気予報:', options=('晴', '曇', '雨', '雪', '快晴', '薄曇'), value='晴')

Button(button_style='success', description='明日の状況をシミュレート', icon='cogs', style=ButtonStyle())

Output()

## アプリケーションコードの概要

このノートブックには、B病院の採血患者数予測に関連する以下の4つのアプリケーションが含まれています。各アプリケーションは、Google Driveに保存されたモデルファイルと設定ファイルを使用します。

1.  **🏥 B病院 日次採血患者数予測アプリ (Google Drive版)**
    *   **概要:** 特定の日の採血患者数を日次で予測します。
    *   **使用ファイル:**
        *   `model_B_daily_finetuned.json` (XGBoostモデル)
        *   `model_B_daily_scaler.joblib` (特徴量スケーラー)
    *   **入力:** 予測日、延べ外来患者数、天気
    *   **主な使用特徴量 (予測用 `create_features_for_pred` 関数内):**
        *   `月`, `週回数`, `前日祝日フラグ`, `total_outpatient_count`, `雨フラグ`, `雪フラグ`, `曜日_月`～`曜日_土`, (`降水量`, `平均気温`, `最高気温`, `最低気温`, `平均湿度`, `平均風速` - UIからは入力しないがモデルに考慮されている可能性のあるカラム)

2.  **🏥 B病院 採血患者数予測アプリ (5営業日・空白スキップ対応版)**
    *   **概要:** 指定した開始日から次の5営業日間の採血患者数を予測します。延べ外来患者数が入力されていない日はスキップします。
    *   **使用ファイル:**
        *   `model_B_daily_finetuned.json` (XGBoostモデル)
        *   `model_B_daily_scaler.joblib` (特徴量スケーラー)
    *   **入力:** 予測開始日、各営業日の延べ外来患者数
    *   **主な使用特徴量 (予測用 `create_features_for_pred` 関数内 - 上記日次アプリと共通):**
        *   `月`, `週回数`, `前日祝日フラグ`, `total_outpatient_count`, `雨フラグ`, `雪フラグ`, `曜日_月`～`曜日_土`, (`降水量`, `平均気温`, `最高気温`, `最低気温`, `平均湿度`, `平均風速`)

3.  **🏥 B病院 時間帯別 採血患者数予測アプリ**
    *   **概要:** 特定の日の時間帯別（30分ごと）の採血患者数（受付人数）を予測し、グラフで表示します。
    *   **使用ファイル:**
        *   `model_B_timeseries_finetuned.json` (XGBoostモデル)
        *   `columns_B_timeseries.json` (特徴量カラムリスト)
    *   **入力:** 予測対象日、延べ外来患者数、天気予報
    *   **主な使用特徴量 (予測ロジック内):**
        *   `hour`, `minute`, `is_first_slot`, `is_second_slot`, `月`, `週回数`, `前日祝日フラグ`, `total_outpatient_count`, `is_holiday`, `雨フラグ`, `雪フラグ`, `天気カテゴリ_晴`～`天気カテゴリ_薄`, `dayofweek_0`～`dayofweek_6`, `lag_30min`, `lag_60min`, `lag_90min`, `rolling_mean_60min`

4.  **🏥 B病院 待ち人数・待ち時間 統合予測アプリ**
    *   **概要:** 特定の日の時間帯別（30分ごと）の予測受付人数、予測待ち人数、予測平均待ち時間をシミュレーションし、グラフと表で表示します。3つのモデルを使用します。
    *   **使用ファイル:**
        *   `model_B_timeseries_finetuned.json` (受付人数予測モデル)
        *   `columns_B_timeseries.json` (受付人数予測用カラムリスト)
        *   `model_B_waittime_30min_ft.json` (待ち時間予測モデル)
        *   `model_B_queue_30min_ft.json` (待ち人数予測モデル)
        *   `columns_B_multi_30min.json` (待ち時間・待ち人数予測用カラムリスト)
    *   **入力:** 予測対象日、延べ外来患者数、天気予報
    *   **主な使用特徴量:**
        *   **受付人数予測用 (上記時間帯別アプリと共通):** `hour`, `minute`, `is_first_slot`, `is_second_slot`, `月`, `週回数`, `前日祝日フラグ`, `total_outpatient_count`, `is_holiday`, `雨フラグ`, `雪フラグ`, `天気カテゴリ_晴`～`天気カテゴリ_薄`, `dayofweek_0`～`dayofweek_6`, `lag_30min`, `lag_60min`, `lag_90min`, `rolling_mean_60min`
        *   **待ち時間・待ち人数予測用:** `hour`, `minute`, `reception_count` (予測受付人数), `queue_at_start_of_slot` (予測待ち人数), `月`, `週回数`, `前日祝日フラグ`, `total_outpatient_count`, `is_holiday`, `天気カテゴリ_晴`～`天気カテゴリ_薄`, `dayofweek_0`～`dayofweek_6`

これらのアプリケーションは、Google Drive上の指定されたパスにモデルファイルと設定ファイルが正しく配置されていることを前提として動作します。